### Imports

In [1]:
import os
os.chdir(os.getcwd() + '/..')
print(os.getcwd())

import data.data as data
import inout.importexport as imp
import numpy as np
import pandas as pd
import xgboost as xgb


/Users/federico/Documents/Repository/RecSys


### Create the complete dataframe

#### Read the recommendations from csv

In [2]:
raw_recs = imp.importcsv('submission/13-12-2018/gxboost25recommendations_12-17-23.csv', check_len=-1)

print(raw_recs[0:2])

[[7, 4492, 15779, 14714, 11257, 20242, 15167, 15908, 3648, 4469, 8749, 17154, 8985, 11641, 5309, 11636, 14400, 4189, 8275, 6615, 8362, 13123, 17780, 20443, 17495, 1063], [25, 4202, 699, 1900, 19134, 14338, 10438, 7077, 12768, 16869, 4899, 800, 2018, 4720, 11633, 10414, 11393, 18567, 11943, 15261, 19390, 12716, 11347, 1422, 7545, 1726]]


#### Explode each row into multiple rows (one per interaction)

In [27]:
recs_tracks = []
for rec in raw_recs:
    playlist_id = rec[0]
    for t in rec[1:]:
        recs_tracks.append([playlist_id, t])
recs_df = pd.DataFrame(recs_tracks, columns=['playlist_id','track_id'])

print(recs_df.head(50))

    playlist_id  track_id
0             7      4492
1             7     15779
2             7     14714
3             7     11257
4             7     20242
5             7     15167
6             7     15908
7             7      3648
8             7      4469
9             7      8749
10            7     17154
11            7      8985
12            7     11641
13            7      5309
14            7     11636
15            7     14400
16            7      4189
17            7      8275
18            7      6615
19            7      8362
20            7     13123
21            7     17780
22            7     20443
23            7     17495
24            7      1063
25           25      4202
26           25       699
27           25      1900
28           25     19134
29           25     14338
30           25     10438
31           25      7077
32           25     12768
33           25     16869
34           25      4899
35           25       800
36           25      2018
37          

#### Append the 'profile_length' column to the recommendation dataframe

In [3]:
target_ids = data.get_target_playlists()
targetURM = data.get_urm_train_1()[target_ids]
user_profile_lengths = np.array(targetURM.sum(axis=1)).flatten()
profile_lengths_df = pd.DataFrame({'playlist_id': target_ids, 'profile_length': user_profile_lengths})

print(profile_lengths_df.head(10))

   playlist_id  profile_length
0            7              28
1           25              13
2           29              18
3           34              24
4           50               8
5           52              16
6           60               8
7           64              12
8           77              29
9           80              18


In [41]:
rec_lengths_df = recs_df.merge(profile_lengths_df, on='playlist_id')
print(rec_lengths_df.head(40))

    playlist_id  track_id  profile_length
0             7      4492              28
1             7     15779              28
2             7     14714              28
3             7     11257              28
4             7     20242              28
5             7     15167              28
6             7     15908              28
7             7      3648              28
8             7      4469              28
9             7      8749              28
10            7     17154              28
11            7      8985              28
12            7     11641              28
13            7      5309              28
14            7     11636              28
15            7     14400              28
16            7      4189              28
17            7      8275              28
18            7      6615              28
19            7      8362              28
20            7     13123              28
21            7     17780              28
22            7     20443         

In [10]:
ohp = pd.get_dummies(rec_lengths_df['playlist_id'])
rec_lengths_df = rec_lengths_df.drop(['playlist_id'], axis=1)

oht = pd.get_dummies(rec_lengths_df['tracks_ids'])
rec_lengths_df = rec_lengths_df.drop(['tracks_ids'], axis=1)

oh1 = pd.concat([rec_lengths_df, ohp, oht], axis=1)

# oh1 = oh1.to_sparse()

oh1

,profile_length,3,6,7,17,18,19,20,25,29,...,20609,20611,20617,20621,20623,20624,20626,20627,20631,20632
0,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,28,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Append the 'label' column 

In [34]:
urm_test = data.get_urm_test_1()
test_labels = []

last_playlist_id = -1
for idx,row in recs_df.iterrows():
    current_playlist_id = row['playlist_id']
    track_id = row['track_id']
    # cache the row of the urm test if same playlist of the previous iteration
    if not current_playlist_id == last_playlist_id:
        # tracks ids in the t row of urm test
        tracks_ids = urm_test.getrow(current_playlist_id).nonzero()[1]
        last_playlist_id = current_playlist_id
    
    test_labels.append(1 if track_id in tracks_ids else 0)

test_labels_df = pd.DataFrame({'label': test_labels})

In [37]:
print(len(test_labels), test_labels[0:30])

250000 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]


In [38]:
oh1 = pd.concat([oh1, test_labels_df], axis=1)

### Split into train and test dataframes

### Train with XGBoost

In [8]:
dtrain = xgb.DMatrix(oh1, [])

KeyboardInterrupt: 